# Cálculo do TF-IDF para os termos dos textos descritivos

## Tokenização dos textos

In [1]:
# Imports
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 

import pandas as pd

#import spacy
import string

In [2]:
## Extraindo frequencias de termos dos textos descritivos do dataset.

In [3]:
df = pd.read_json('data_files/merged_datasets.json', orient='split')

In [8]:
corpus = df['Text']

In [10]:
# Apagando a variável df porque não será mais utilizada.
del df

In [11]:
# Quebrando cada texto em uma lista de sentenças.
corpus_sent_tokenized = [sent_tokenize(texto) for texto in corpus]

In [12]:
# Apagando a variável corpus porque não será mais utilizada.
del corpus

In [15]:
# Quebrando as sentenças em palavras, de modo que cada sentença seja transformada em lista de palavras.
corpus_word_tokenized = [[word_tokenize(sent) for sent in text] for text in corpus_sent_tokenized]

In [17]:
# Apagando a variável corpus_sent_tokenized porque não será mais utilizada
del corpus_sent_tokenized

## Remoção das stopwords e pontuações

In [18]:
english_stops = set(stopwords.words('english'))

In [19]:
def remove_stopwords_e_pontuacao(lista_words):
    tokens_limpos = [word for word in lista_words if (word not in english_stops and word not in string.punctuation)]
    return tokens_limpos

In [20]:
corpus_limpo = [[remove_stopwords_e_pontuacao(sent) for sent in text] for text in corpus_word_tokenized]

In [23]:
# Apagando a variável corpus_word_tokenized porque não será mais utilizada.
del corpus_word_tokenized